In [50]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import matplotlib.pyplot as plt
import numpy as np

In [67]:
# Read in cinema locations between 1948 to 1971 - Cinemas1948.csv 
csv_path = "../../Resources/Cinemas1948.csv"
Cinemas1948_df = pd.read_csv(csv_path, engine='python')

# Display Dataframe
Cinemas1948_df.head()

,id,latitude,longitude,australian_states_id,locality_types_id,film_weekly_cinema_types_id,street,suburb,postcode,cinema_name,exhibitor_name,capacity,shortname,description,description.1,helper column
0,1,-43.431721,146.974298,8,3,3,NaN,Southport,7109.0,Cruickshank & Douglas,Cruickshank & Douglas,NaN,TAS,Touring Circuit,Rural,0
1,2,-43.431282,146.974205,8,3,3,NaN,Southport,7109.0,Frank Ward,"Auvergne Ave., New Town, Hobart",NaN,TAS,Touring Circuit,Rural,1
2,3,-43.315922,147.242881,8,3,1,NaN,"Aloonah, South Bruny Island",7150.0,Aloonah,T. V. Murphy,150.0,TAS,Indoor Cinema,Rural,0
3,4,-43.315877,147.242850,8,3,1,NaN,"Alonnah, South Bruny Island",7150.0,Pictures,T. V. Murphy,NaN,TAS,Indoor Cinema,Rural,1
4,5,-43.313472,147.015336,8,3,1,NaN,Dover,7117.0,Hall,Ryan Brothers,320.0,TAS,Indoor Cinema,Rural,0


In [68]:
# Drop australian_states_id, locality_types_id, film_weekly_cinema_types_id, exhibitor_name, capacity, shortname, description.1, id
Cinemas1948_df = Cinemas1948_df[["cinema_name", "street", "suburb", "postcode", "description", "latitude", "longitude", "shortname"]]

# Rename shortname to state, description to cinema_type
Cinemas1948_df = Cinemas1948_df.rename(columns={"shortname":"state", "description":"cinema_type"})

# Drop .0 in postcode column
Cinemas1948_df = Cinemas1948_df.astype(str).replace(r'\.0$', '', regex=True)

# Dusplay dataframe
Cinemas1948_df.head()

,cinema_name,street,suburb,postcode,cinema_type,latitude,longitude,state
0,Cruickshank & Douglas,nan,Southport,7109,Touring Circuit,-43.431721,146.974298,TAS
1,Frank Ward,nan,Southport,7109,Touring Circuit,-43.431282,146.974205,TAS
2,Aloonah,nan,"Aloonah, South Bruny Island",7150,Indoor Cinema,-43.315922,147.242881,TAS
3,Pictures,nan,"Alonnah, South Bruny Island",7150,Indoor Cinema,-43.3158772,147.2428501,TAS
4,Hall,nan,Dover,7117,Indoor Cinema,-43.313472,147.01533600000002,TAS


In [69]:
# Save to CSV
Cinemas1948_df.to_csv("../../Resources/Cinemas1948_clean.csv", index=False, header=True)


In [10]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Visit website
url = 'http://cinematreasures.org/theaters/australia?status=all'
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\austr\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [31]:
# Use find all function to navigate to data
results = soup.find_all('tr', class_='location odd theater')


In [32]:
# Display results
results

[<tr class="location odd theater" data-address="1-3 Rose Street&lt;br&gt;Melbourne, VIC 3156&lt;br&gt;Australia&lt;br&gt;+61-03-9758-3964&lt;br&gt;" data-category="open" data-id="31849" data-lat="-37.8941003" data-lng="145.3107163" data-name="1812 Theatre" data-photo="http://photos.cinematreasures.org/production/photos/59887/1351700028/thumb.JPG?1351700028">
 <td class="name">
 <a class="map-link" href="/theaters/31849"><img alt="1812 Theatre" height="48" src="http://photos.cinematreasures.org/production/photos/59887/1351700028/thumb.JPG?1351700028" width="48"/></a>
 <a href="/theaters/31849">1812 Theatre</a>
 </td>
 <td class="location">
                 Melbourne, Australia
               </td>
 <td class="status">
                 Open
               </td>
 <td class="screens">
                 1
               </td>
 </tr>,
 <tr class="location odd theater" data-address="91 Hindley Street&lt;br&gt;Adelaide, SA 5000&lt;br&gt;Australia&lt;br&gt;+61-08-8233-6233&lt;br&gt;" data-catego

In [33]:
# Extract cinema name, address, status, latitude and longitude
lat = results[0]['data-lat']
lng = results[0]['data-lng']
cinema_name = results[0]['data-name']
cinema_address = results[0]['data-address']
cinema_status = results[0]['data-category']

In [47]:
# Create a list to store variables
list = []

for x in range(1, 75):

    # Loop through variables
    for result in results:
        cinema_name = result['data-name']
        cinema_address = result['data-address']
        lng = result['data-lng']
        lat = result['data-lat']
        cinema_status = result['data-category']

        # Create a dictionary 
        post = {
            'Cinema name': cinema_name,
            'Cinema address': cinema_address,
            'long': lng,
            'lat': lat,
            'Cinema Status': cinema_status
        }

        # Append data to the list
        list.append(post)

    # Create loop to click next button to get to next page
    try:
        for lists in list:
            browser.links.find_by_partial_text('next').click()
    except: 
        print("Scraping Complete")

Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping Complete
Scraping C

In [48]:
# Display list of arrays
list

[{'Cinema name': '1812 Theatre',
  'Cinema address': '1-3 Rose Street<br>Melbourne, VIC 3156<br>Australia<br>+61-03-9758-3964<br>',
  'long': '145.3107163',
  'lat': '-37.8941003',
  'Cinema Status': 'open'},
 {'Cinema name': 'ASO Grainger Studio',
  'Cinema address': '91 Hindley Street<br>Adelaide, SA 5000<br>Australia<br>+61-08-8233-6233<br>',
  'long': '138.595823',
  'lat': '-34.92337',
  'Cinema Status': 'closed'},
 {'Cinema name': 'Academy Twin Cinemas',
  'Cinema address': '3A Oxford Street<br>Sydney, NSW 2021<br>Australia<br>',
  'long': '151.21949',
  'lat': '-33.881906',
  'Cinema Status': 'closed'},
 {'Cinema name': 'Ace Cinemas Rockingham',
  'Cinema address': '1 Syren Street<br>Rockingham, WA 6168<br>Australia<br>+61-8-9550-9000<br>',
  'long': '115.7483092',
  'lat': '-32.288672',
  'Cinema Status': 'open'},
 {'Cinema name': 'Ace Midland Gate Cinema',
  'Cinema address': 'Great Eastern Highway<br>Midland, WA 6056<br>Australia<br>+61(0)8-9250-2620<br>',
  'long': '116.0119

In [71]:
# Convert list into a dataframe
df = pd.DataFrame.from_records(list)

In [72]:
df

,Cinema name,Cinema address,long,lat,Cinema Status
0,1812 Theatre,"1-3 Rose Street<br>Melbourne, VIC 3156<br>Aust...",145.3107163,-37.8941003,open
1,ASO Grainger Studio,"91 Hindley Street<br>Adelaide, SA 5000<br>Aust...",138.595823,-34.92337,closed
2,Academy Twin Cinemas,"3A Oxford Street<br>Sydney, NSW 2021<br>Austra...",151.21949,-33.881906,closed
3,Ace Cinemas Rockingham,"1 Syren Street<br>Rockingham, WA 6168<br>Austr...",115.7483092,-32.288672,open
4,Ace Midland Gate Cinema,"Great Eastern Highway<br>Midland, WA 6056<br>A...",116.0119137,-31.8916168,open
...,...,...,...,...,...
883,Adelphi Theatre,"367 Nicholson Street<br>Melbourne, VIC 3054<br...",144.9759243,-37.7913005,closed
884,Agricultural Hall,"21 Victoria Road<br>Mingenew, WA 6522<br>Austr...",115.4417026,-29.1924326,closed
885,Albany Entertainment Centre,"2 Toll Place<br>Albany, WA 6330<br>Australia<br>",117.8859423,-35.0292594,open
886,Alexander Theatre,"Wellington Road<br>Melbourne, VIC 3800<br>Aust...",145.1340467,-37.9153572,closed


In [73]:
# Save to CSV
df.to_csv("../../Resources/Cinemas2022_clean.csv", index=False, header=True)
